In [1]:
import cv2
import mediapipe as mp
from flask import Flask, jsonify
import threading
import time

# Flaskアプリの初期化
app = Flask(__name__)

# MediaPipeのPoseモジュールをセットアップ
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    model_complexity=1,  # 精度重視のモデル (0: 簡易版, 1: 高精度)
    smooth_landmarks=True,  # ランドマークのスムージングを有効にする
    enable_segmentation=False,  # セグメンテーション機能は無効
    smooth_segmentation=True,  # セグメンテーションのスムージング
    min_detection_confidence=0.5,  # 最小検出信頼度 (0.0〜1.0)
    min_tracking_confidence=0.5   # 最小追跡信頼度 (0.0〜1.0)
)

# MediaPipeの描画ユーティリティ
mp_drawing = mp.solutions.drawing_utils

# グローバルな変数でカメラ映像を保持
frame = None
pose_landmarks = None

# カメラの初期化
cap = cv2.VideoCapture(0)

def capture_frame():
    global frame, pose_landmarks
    while cap.isOpened():
        ret, current_frame = cap.read()
        if not ret:
            break

        # OpenCVのBGRをRGBに変換（MediaPipeはRGBを使用）
        frame_rgb = cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB)

        # ポーズ検出を実行
        results = pose.process(frame_rgb)

        # ランドマークの座標をグローバル変数に保存
        if results.pose_landmarks:
            pose_landmarks = results.pose_landmarks.landmark

        # 最新のフレームを保持
        frame = current_frame

        # 骨格検出結果を描画
        if results.pose_landmarks:
            # ポーズランドマークを描画（関節を線で結ぶ）
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # ランドマークの座標を表示
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                # 座標 (x, y) を画面に表示 (xは幅、yは高さの割合)
                h, w, c = frame.shape  # 画像の高さ、幅、チャンネル数
                x, y = int(landmark.x * w), int(landmark.y * h)  # 座標をピクセルに変換
                # 座標をコンソールに出力
                print(f"Landmark {idx}: ({x}, {y})")
                # 座標をフレーム上に表示
                cv2.putText(frame, f"{idx}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


        

        # 出力ウィンドウに表示
        cv2.imshow('Pose Detection', frame)

        # 'q'キーでウィンドウを閉じる
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # 少し待つ（無限ループ防止）
        time.sleep(0.05)

# 座標情報をAPIで返すエンドポイント
@app.route('/pose', methods=['GET'])
def get_pose():
    global pose_landmarks
    if pose_landmarks:
        landmarks_data = []
        for idx, landmark in enumerate(pose_landmarks):
            landmarks_data.append({
                'landmark_id': idx,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z
            })
        return jsonify(landmarks_data)
    else:
        return jsonify({"error": "No pose landmarks detected"}), 400

# Flaskを別スレッドで起動
if __name__ == '__main__':
    # 別スレッドでカメラのフレームキャプチャを開始
    capture_thread = threading.Thread(target=capture_frame, daemon=True)
    capture_thread.start()
    
    # Flaskアプリを起動
    app.run(debug=False, use_reloader=False, host='0.0.0.0', port=5000)
    
    # カメラをリリースしてウィンドウを閉じる
    cap.release()
    cv2.destroyAllWindows()



 * Serving Flask app '__main__'
 * Debug mode: off


I0000 00:00:1739536266.870028   34977 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1739536266.876920   35074 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: AMD Radeon Graphics (radeonsi, gfx1103_r1, LLVM 17.0.6, DRM 3.57, 6.8.0-52-lowlatency)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.3.10:5000
Press CTRL+C to quit
W0000 00:00:1739536267.080984   35059 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739536267.176920   35058 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739536267.556270   35054 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only s

Landmark 0: (309, 403)
Landmark 1: (322, 377)
Landmark 2: (331, 376)
Landmark 3: (339, 376)
Landmark 4: (295, 378)
Landmark 5: (287, 377)
Landmark 6: (279, 376)
Landmark 7: (355, 376)
Landmark 8: (270, 379)
Landmark 9: (326, 422)
Landmark 10: (296, 423)
Landmark 11: (426, 464)
Landmark 12: (213, 463)
Landmark 13: (478, 599)
Landmark 14: (145, 599)
Landmark 15: (466, 707)
Landmark 16: (137, 697)
Landmark 17: (478, 747)
Landmark 18: (124, 733)
Landmark 19: (452, 745)
Landmark 20: (145, 728)
Landmark 21: (443, 729)
Landmark 22: (154, 715)
Landmark 23: (382, 722)
Landmark 24: (247, 722)
Landmark 25: (384, 924)
Landmark 26: (258, 923)
Landmark 27: (381, 1104)
Landmark 28: (260, 1104)
Landmark 29: (386, 1134)
Landmark 30: (259, 1136)
Landmark 31: (352, 1172)
Landmark 32: (276, 1167)
Landmark 0: (307, 402)
Landmark 1: (321, 377)
Landmark 2: (331, 376)
Landmark 3: (339, 375)
Landmark 4: (293, 377)
Landmark 5: (285, 377)
Landmark 6: (278, 376)
Landmark 7: (354, 375)
Landmark 8: (270, 378)
Landm

127.0.0.1 - - [14/Feb/2025 21:31:34] "GET /pose HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2025 21:31:34] "GET /pose HTTP/1.1" 200 -


Landmark 0: (308, 409)
Landmark 1: (323, 382)
Landmark 2: (333, 381)
Landmark 3: (341, 380)
Landmark 4: (295, 383)
Landmark 5: (287, 382)
Landmark 6: (280, 381)
Landmark 7: (356, 378)
Landmark 8: (274, 377)
Landmark 9: (328, 428)
Landmark 10: (295, 429)
Landmark 11: (428, 470)
Landmark 12: (208, 463)
Landmark 13: (475, 603)
Landmark 14: (142, 604)
Landmark 15: (463, 701)
Landmark 16: (139, 705)
Landmark 17: (472, 738)
Landmark 18: (128, 739)
Landmark 19: (453, 734)
Landmark 20: (149, 737)
Landmark 21: (446, 718)
Landmark 22: (158, 724)
Landmark 23: (380, 744)
Landmark 24: (242, 742)
Landmark 25: (386, 965)
Landmark 26: (257, 960)
Landmark 27: (393, 1168)
Landmark 28: (273, 1166)
Landmark 29: (397, 1203)
Landmark 30: (275, 1202)
Landmark 31: (367, 1235)
Landmark 32: (291, 1229)
Landmark 0: (307, 412)
Landmark 1: (322, 384)
Landmark 2: (332, 383)
Landmark 3: (340, 382)
Landmark 4: (294, 386)
Landmark 5: (286, 385)
Landmark 6: (280, 384)
Landmark 7: (356, 379)
Landmark 8: (274, 378)
Landm

...

Landmark 0: (362, 285)
Landmark 1: (379, 266)
Landmark 2: (387, 267)
Landmark 3: (396, 269)
Landmark 4: (353, 262)
Landmark 5: (344, 261)
Landmark 6: (334, 260)
Landmark 7: (402, 278)
Landmark 8: (319, 266)
Landmark 9: (369, 313)
Landmark 10: (340, 309)
Landmark 11: (446, 385)
Landmark 12: (225, 396)
Landmark 13: (470, 535)
Landmark 14: (140, 491)
Landmark 15: (493, 659)
Landmark 16: (107, 663)
Landmark 17: (511, 700)
Landmark 18: (89, 706)
Landmark 19: (502, 705)
Landmark 20: (115, 710)
Landmark 21: (488, 692)
Landmark 22: (126, 696)
Landmark 23: (404, 700)
Landmark 24: (250, 699)
Landmark 25: (397, 968)
Landmark 26: (258, 964)
Landmark 27: (402, 1217)
Landmark 28: (267, 1211)
Landmark 29: (403, 1253)
Landmark 30: (262, 1247)
Landmark 31: (387, 1290)
Landmark 32: (294, 1283)
Landmark 0: (362, 285)
Landmark 1: (378, 266)
Landmark 2: (387, 267)
Landmark 3: (396, 269)
Landmark 4: (353, 262)
Landmark 5: (343, 261)
Landmark 6: (334, 261)
Landmark 7: (402, 278)
Landmark 8: (319, 266)
Landma

127.0.0.1 - - [14/Feb/2025 21:34:56] "GET /pose HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2025 21:34:56] "GET /pose HTTP/1.1" 200 -


Landmark 0: (361, 285)
Landmark 1: (378, 266)
Landmark 2: (386, 267)
Landmark 3: (396, 269)
Landmark 4: (352, 262)
Landmark 5: (343, 261)
Landmark 6: (334, 261)
Landmark 7: (402, 278)
Landmark 8: (319, 266)
Landmark 9: (369, 313)
Landmark 10: (340, 310)
Landmark 11: (446, 385)
Landmark 12: (225, 396)
Landmark 13: (469, 536)
Landmark 14: (140, 491)
Landmark 15: (493, 661)
Landmark 16: (106, 664)
Landmark 17: (511, 702)
Landmark 18: (87, 706)
Landmark 19: (502, 706)
Landmark 20: (114, 710)
Landmark 21: (488, 694)
Landmark 22: (125, 696)
Landmark 23: (404, 700)
Landmark 24: (250, 699)
Landmark 25: (397, 968)
Landmark 26: (257, 964)
Landmark 27: (401, 1217)
Landmark 28: (266, 1210)
Landmark 29: (402, 1253)
Landmark 30: (261, 1247)
Landmark 31: (385, 1290)
Landmark 32: (292, 1283)
Landmark 0: (361, 285)
Landmark 1: (378, 266)
Landmark 2: (386, 267)
Landmark 3: (396, 269)
Landmark 4: (352, 262)
Landmark 5: (343, 261)
Landmark 6: (334, 261)
Landmark 7: (402, 278)
Landmark 8: (319, 267)
Landma

127.0.0.1 - - [14/Feb/2025 21:34:56] "GET /pose HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2025 21:34:56] "GET /pose HTTP/1.1" 200 -


Landmark 0: (361, 285)
Landmark 1: (378, 266)
Landmark 2: (386, 267)
Landmark 3: (395, 269)
Landmark 4: (352, 262)
Landmark 5: (343, 261)
Landmark 6: (333, 261)
Landmark 7: (402, 278)
Landmark 8: (319, 267)
Landmark 9: (369, 314)
Landmark 10: (340, 310)
Landmark 11: (446, 385)
Landmark 12: (224, 396)
Landmark 13: (469, 537)
Landmark 14: (141, 493)
Landmark 15: (493, 662)
Landmark 16: (107, 667)
Landmark 17: (511, 703)
Landmark 18: (88, 711)
Landmark 19: (502, 707)
Landmark 20: (115, 715)
Landmark 21: (488, 695)
Landmark 22: (126, 700)
Landmark 23: (403, 701)
Landmark 24: (249, 700)
Landmark 25: (396, 968)
Landmark 26: (257, 964)
Landmark 27: (400, 1217)
Landmark 28: (265, 1211)
Landmark 29: (401, 1253)
Landmark 30: (261, 1248)
Landmark 31: (384, 1290)
Landmark 32: (291, 1283)
Landmark 0: (361, 285)
Landmark 1: (378, 266)
Landmark 2: (386, 267)
Landmark 3: (395, 269)
Landmark 4: (352, 262)
Landmark 5: (343, 261)
Landmark 6: (333, 261)
Landmark 7: (402, 278)
Landmark 8: (319, 267)
Landma